### Chapter 4: Subqueries

**1. Identify legacy customers whose initial order date is older than the average initial order date across the customer base.**

**Functional Specification**
* Join Sales.Customers (c) to Sales.Orders (o).
* Find the first order date for each customer using MIN(o.OrderDate) and GROUP BY.
* Use a nested subquery to calculate the average of all first order dates.
* Filter the results in the HAVING clause to keep only customers whose first order date is earlier than the calculated average.

In [ ]:
SELECT
    c.CustomerID,
    c.CustomerName,
    MIN(o.OrderDate) AS FirstOrderDate
FROM
    Sales.Customers AS c
JOIN
    Sales.Orders AS o ON c.CustomerID = o.CustomerID
GROUP BY
    c.CustomerID, c.CustomerName
HAVING
    CAST(CAST(MIN(o.OrderDate) AS DATETIME) AS FLOAT) < (
        SELECT AVG(CAST(CAST(FirstOrderDates.FirstDate AS DATETIME) AS float))
        FROM (
            SELECT MIN(OrderDate) AS FirstDate
            FROM Sales.Orders
            GROUP BY CustomerID
        ) AS FirstOrderDates
    );

**2. Perform a market analysis to identify products frequently purchased along with the 'USB missile launcher (Update)'.**

**Functional Specification**
* Use a subquery to find the StockItemID for 'USB missile launcher (Update)'.
* Use another subquery with the IN predicate to get all OrderIDs that contain this item.
* Select all distinct StockItemName from those orders.
* Exclude the 'USB missile launcher (Update)' itself from the final results.

In [ ]:
SELECT DISTINCT
    si.StockItemName
FROM
    Sales.OrderLines AS ol
JOIN
    Warehouse.StockItems AS si ON ol.StockItemID = si.StockItemID
WHERE
    ol.OrderID IN (
        SELECT OrderID
        FROM Sales.OrderLines
        WHERE StockItemID = (
            SELECT StockItemID FROM Warehouse.StockItems WHERE StockItemName = N'USB missile launcher (Update)'
        )
    )
    AND si.StockItemName <> N'USB missile launcher (Update)';

**3. Identify salespeople who have successfully sold to every existing customer category.**

**Functional Specification**
* Join People to Orders and Customers to link salespeople to customer categories.
* Group the results by salesperson (p.FullName).
* Use a subquery in the HAVING clause to get the total count of all customer categories.
* Compare the count of distinct categories for each salesperson to the total count; only return those who match.

In [ ]:
SELECT
    p.FullName AS Salesperson
FROM
    Application.People AS p
JOIN
    Sales.Orders AS o ON p.PersonID = o.SalespersonPersonID
JOIN
    Sales.Customers AS c ON o.CustomerID = c.CustomerID
GROUP BY
    p.FullName
HAVING
    COUNT(DISTINCT c.CustomerCategoryID) = (SELECT COUNT(*) FROM Sales.CustomerCategories);

**4. Isolate suppliers who exclusively provide ambient (non chiller) stock items.**

**Functional Specification**
* Use a subquery with the NOT IN predicate.
* The subquery creates a list of all distinct SupplierIDs that are linked to a chiller stock item (IsChillerStock = 1).
* The outer query selects all suppliers from Purchasing.Suppliers whose SupplierID is not in this list.

In [ ]:
SELECT
    SupplierID,
    SupplierName
FROM
    Purchasing.Suppliers
WHERE
    SupplierID NOT IN (
        SELECT DISTINCT SupplierID
        FROM Warehouse.StockItems
        WHERE IsChillerStock = 1
    );

**5. List products priced above average and show their price difference.**

**Functional Specification**
* Use a scalar subquery to calculate the single average UnitPrice from Warehouse.StockItems.
* Filter the StockItems table to select rows where UnitPrice is greater than this average.
* In the SELECT list, calculate the difference between the item's UnitPrice and the average price.
* Order the results to show the most expensive items first.

In [ ]:
SELECT
    StockItemName,
    UnitPrice,
    (UnitPrice - (SELECT AVG(UnitPrice) FROM Warehouse.StockItems)) AS PriceDifferenceFromAverage
FROM
    Warehouse.StockItems
WHERE
    UnitPrice > (SELECT AVG(UnitPrice) FROM Warehouse.StockItems)
ORDER BY
    UnitPrice DESC;

### Chapter 5: Table Expressions

**6. Determine the top performing salesperson by total revenue for each fiscal year.**

**Functional Specification**
* Create a CTE named SalesRanks.
* Inside the CTE, join Invoices, InvoiceLines, and People.
* Group by year and salesperson to calculate total revenue.
* Use the ROW_NUMBER() window function to rank salespeople within each year by their revenue.
* Select from the CTE where the rank is 1.

In [ ]:
WITH SalesRanks AS (
    SELECT
        YEAR(i.InvoiceDate) AS SalesYear,
        p.FullName AS Salesperson,
        SUM(il.Quantity * il.UnitPrice) AS TotalRevenue,
        ROW_NUMBER() OVER(PARTITION BY YEAR(i.InvoiceDate) ORDER BY SUM(il.Quantity * il.UnitPrice) DESC) AS RankNum
    FROM
        Sales.Invoices AS i
    JOIN
        Sales.InvoiceLines AS il ON i.InvoiceID = il.InvoiceID
    JOIN
        Application.People AS p ON i.SalespersonPersonID = p.PersonID
    GROUP BY
        YEAR(i.InvoiceDate), p.FullName
)
SELECT
    sr.SalesYear,
    sr.Salesperson,
    sr.TotalRevenue
FROM
    SalesRanks AS sr
WHERE
    sr.RankNum = 1
ORDER BY
    sr.SalesYear;

**7. Calculate the customer lifecycle duration by identifying the first and last order dates for each customer.**

**Functional Specification**
* Create a derived table aliased as 'cod'.
* Inside the derived table, group the Sales.Orders table by CustomerID.
* Use MIN(OrderDate) and MAX(OrderDate) to find the first and last order dates for each customer.
* Join the main Customers table to this derived table on CustomerID to link the dates to customer names.

In [ ]:
SELECT
    c.CustomerName,
    cod.FirstOrder,
    cod.LastOrder
FROM
    Sales.Customers AS c
JOIN (
    SELECT
        CustomerID,
        MIN(OrderDate) AS FirstOrder,
        MAX(OrderDate) AS LastOrder
    FROM
        Sales.Orders
    GROUP BY
        CustomerID
) AS cod ON c.CustomerID = cod.CustomerID
ORDER BY
    cod.LastOrder DESC;

**8. Identify the top 3 revenue generating products within each customer category.**

**Functional Specification**
* Use CROSS APPLY to run a correlated subquery for each row in Sales.CustomerCategories.
* The subquery joins StockItems, OrderLines, Orders, and Customers.
* It filters for sales belonging to the current customer category.
* It groups by StockItemName to calculate total revenue per product and selects the TOP 3.
* The final result combines each category with its top 3 products.

In [ ]:
SELECT
    cc.CustomerCategoryName,
    TopProducts.StockItemName,
    TopProducts.TotalRevenue
FROM
    Sales.CustomerCategories AS cc
CROSS APPLY (
    SELECT TOP (3)
        si.StockItemName,
        SUM(ol.Quantity * ol.UnitPrice) AS TotalRevenue
    FROM
        Warehouse.StockItems AS si
    JOIN
        Sales.OrderLines AS ol ON si.StockItemID = ol.StockItemID
    JOIN
        Sales.Orders AS o ON ol.OrderID = o.OrderID
    JOIN
        Sales.Customers AS c ON o.CustomerID = c.CustomerID
    WHERE
        c.CustomerCategoryID = cc.CustomerCategoryID
    GROUP BY
        si.StockItemName
    ORDER BY
        SUM(ol.Quantity * ol.UnitPrice) DESC
) AS TopProducts;

**9. Segment the customer base into value tiers ('High', 'Medium', 'Low') based on total historical spending.**

**Functional Specification**
* Create a CTE named CustomerSpending.
* Inside the CTE, join Invoices and InvoiceLines and group by CustomerID to calculate total spending for each customer.
* Join the main Customers table to the CTE.
* Use a CASE expression in the SELECT list to assign a 'CustomerTier' based on the total spending amount.

In [ ]:
WITH CustomerSpending AS (
    SELECT
        i.CustomerID,
        SUM(il.Quantity * il.UnitPrice) AS TotalSpent
    FROM
        Sales.Invoices AS i
    JOIN
        Sales.InvoiceLines AS il ON i.InvoiceID = il.InvoiceID
    GROUP BY
        i.CustomerID
)
SELECT
    c.CustomerName,
    cs.TotalSpent,
    CASE
        WHEN cs.TotalSpent > 50000 THEN 'High Value'
        WHEN cs.TotalSpent > 10000 THEN 'Medium Value'
        ELSE 'Low Value'
    END AS CustomerTier
FROM
    Sales.Customers AS c
JOIN
    CustomerSpending AS cs ON c.CustomerID = cs.CustomerID
ORDER BY
    cs.TotalSpent DESC;

**10. For each invoice, provide a breakdown of the total value of 'Chiller' vs. 'Ambient' items.**

**Functional Specification**
* Create a CTE named InvoiceComposition.
* Inside the CTE, join InvoiceLines and StockItems.
* Use a CASE expression on si.IsChillerStock to create a new column, ItemCategory ('Chiller' or 'Ambient').
* Group by InvoiceID and the new ItemCategory to sum the value for each type within each invoice.
* The final query selects from the CTE to display the categorized breakdown.

In [ ]:
WITH InvoiceComposition AS (
    SELECT
        il.InvoiceID,
        CASE
            WHEN si.IsChillerStock = 1 THEN 'Chiller Items'
            ELSE 'Ambient Items'
        END AS ItemCategory,
        SUM(il.Quantity * il.UnitPrice) AS TotalValue
    FROM
        Sales.InvoiceLines AS il
    JOIN
        Warehouse.StockItems AS si ON il.StockItemID = si.StockItemID
    GROUP BY
        il.InvoiceID,
        CASE WHEN si.IsChillerStock = 1 THEN 'Chiller Items' ELSE 'Ambient Items' END
)
SELECT
    ic.InvoiceID,
    ic.ItemCategory,
    ic.TotalValue
FROM
    InvoiceComposition AS ic
ORDER BY
    ic.InvoiceID, ic.ItemCategory;